In [ ]:
from dataset import ClipGPTFlickr8kDataset

In [ ]:
data = './data/oscar_split_ViT-B_32_train_arabic.pkl'
prefix_length = 512

dataset = ClipGPTFlickr8kDataset(data, prefix_length, normalize_prefix=True)


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import pickle
with open(data, 'rb') as f:
    all_data = pickle.load(f)
captions_raw = all_data["captions"]
tokenizer = AutoTokenizer.from_pretrained("akhooli/gpt2-small-arabic")
print("Data size is %0d" % len(all_data["clip_embedding"]))

In [10]:
import torch
captions_tokens = []
caption2embedding = []
max_seq_len = 0
for caption in captions_raw:
    captions_tokens.append(torch.tensor(tokenizer.encode(caption['caption'][0]), dtype=torch.int64))
    

array([1, 1, 0, 2, 0, 1, 0, 0, 1, 1, 1, 2, 2, 0, 1, 0, 0, 1, 1, 2])